# NYC Flights Analysis

Data Transformation with R Programming

DataRockie: Data Science Bootcamp

## Data Transformation

In [ ]:
install.packages("nycflights13")

library(dplyr)
library(readr)
library(tidyverse)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.0     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ stringr 1.4.1
✔ tidyr   1.3.0     ✔ forcats 0.5.2

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()



In [ ]:
# Preparing Data

airlines <- read.csv("airlines.csv")
flights  <- read.csv("flights.csv")
airports <- read.csv("airports.csv")
planes   <- read.csv("planes.csv")
weather <- read.csv("weather.csv")

In [ ]:
## Confirm structure of DataFrame

glimpse(flights)
cat("\n")
glimpse(airlines)
cat("\n")
glimpse(airports)
cat("\n")
glimpse(planes)
cat("\n")
glimpse(weather)

Rows: 336,776
Columns: 19
$ year           <int> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2…
$ month          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ day            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ dep_time       <int> 517, 533, 542, 544, 554, 554, 555, 557, 557, 558, 558, …
$ sched_dep_time <int> 515, 529, 540, 545, 600, 558, 600, 600, 600, 600, 600, …
$ dep_delay      <int> 2, 4, 2, -1, -6, -4, -5, -3, -3, -2, -2, -2, -2, -2, -1…
$ arr_time       <int> 830, 850, 923, 1004, 812, 740, 913, 709, 838, 753, 849,…
$ sched_arr_time <int> 819, 830, 850, 1022, 837, 728, 854, 723, 846, 745, 851,…
$ arr_delay      <int> 11, 20, 33, -18, -25, 12, 19, -14, -8, 8, -2, -3, 7, -1…
$ carrier        <chr> "UA", "UA", "AA", "B6", "DL", "UA", "B6", "EV", "B6", "…
$ flight         <int> 1545, 1714, 1141, 725, 461, 1696, 507, 5708, 79, 301, 4…
$ tailnum        <chr> "N14228", "N24211", "N619AA", "N804JB", "N668DN", "N394…
$ origin      

### 01 Which airline has total longest flight distance in August 2013?

In [ ]:
(flights %>%
  filter(year == 2013, month == 8) %>%
  group_by(carrier) %>%
  summarise(sum_distance = sum(distance)) %>%
  arrange(desc(sum_distance)) %>%
  left_join(airlines, by = "carrier") -> df1) %>%
  head(5)

carrier,sum_distance,name
<chr>,<int>,<chr>
UA,8162260,United Air Lines Inc.
DL,5326470,Delta Air Lines Inc.
B6,5278235,JetBlue Airways
AA,3787864,American Airlines Inc.
EV,2651178,ExpressJet Airlines Inc.


### 02 Top 5 airline operated in Quarter 3 of 2013 (July - September, 2013)

In [ ]:
(flights %>%
    drop_na() %>%
    filter(month %in% c(7, 8, 9)) %>%
    group_by(carrier) %>%
    count(carrier) %>%
    rename(operate_counts = n) %>%
    arrange(desc(operate_counts)) %>%
    left_join(airlines %>%
        rename(carrier_name = name), by = "carrier") %>%
    select(carrier_name, operate_counts) -> df2) %>%
    head(5)

Adding missing grouping variables: `carrier`


carrier,carrier_name,operate_counts
<chr>,<chr>,<int>
UA,United Air Lines Inc.,14692
B6,JetBlue Airways,14113
EV,ExpressJet Airlines Inc.,13188
DL,Delta Air Lines Inc.,12356
AA,American Airlines Inc.,8156


### 03 The most frequently used manufacturer airplane in 2013, and describe the aircraft manufacturer and model.

In [ ]:
#Check N/A of model/manufacturer/tail number of planes dataframe

check_na <- function(col) {
  sum(is.na(col))
}

apply(planes, MARGIN=2, check_na)

tailnum         year         type manufacturer        model      engines 
           0           70            0            0            0            0 
       seats        speed       engine 
           0         3299            0

In [ ]:
(flights %>%
    drop_na() %>%
    select(tailnum, carrier) %>%
    left_join(planes %>%
        select(tailnum, manufacturer, model), by="tailnum") %>%
    drop_na() %>%
    count(model, manufacturer) %>%
    rename(operated_count = n) %>%
    arrange(desc(operated_count)) -> df3) %>%
    head(10)

,model,manufacturer,operated_count
,<chr>,<chr>,<int>
1,A320-232,AIRBUS,30972
2,EMB-145LR,EMBRAER,26475
3,ERJ 190-100 IGW,EMBRAER,23481
4,A320-232,AIRBUS INDUSTRIE,14465
5,737-824,BOEING,13773
6,EMB-145XR,EMBRAER,13312
7,CL-600-2D24,BOMBARDIER INC,11656
8,737-7H4,BOEING,10227
9,757-222,BOEING,9126


### 04 The highest demand route with name of origin and destination airport

In [ ]:
#flights %>%
#   distinct(origin, dest)

(flights %>%
    count(origin,dest) %>%
    arrange(desc(n)) %>%
    left_join(airports %>%
        select(faa, name), by = c("origin" = "faa")) %>%
    rename(name_origin = name) %>%
    left_join(airports %>%
        select(faa, name), by = c("dest" = "faa")) %>%
    rename(name_dest = name) %>%
    select(name_origin, origin, name_dest, dest, n) -> df44) %>%
    head(10) 

,name_origin,origin,name_dest,dest,n
,<chr>,<chr>,<chr>,<chr>,<int>
1,John F Kennedy Intl,JFK,Los Angeles Intl,LAX,11262
2,La Guardia,LGA,Hartsfield Jackson Atlanta Intl,ATL,10263
3,La Guardia,LGA,Chicago Ohare Intl,ORD,8857
4,John F Kennedy Intl,JFK,San Francisco Intl,SFO,8204
5,La Guardia,LGA,Charlotte Douglas Intl,CLT,6168
6,Newark Liberty Intl,EWR,Chicago Ohare Intl,ORD,6100
7,John F Kennedy Intl,JFK,General Edward Lawrence Logan Intl,BOS,5898
8,La Guardia,LGA,Miami Intl,MIA,5781
9,John F Kennedy Intl,JFK,Orlando Intl,MCO,5464


### 05 Top 5 airlines and destinations with later than 60-minute arrival

In [ ]:
late_arr_flights <- flights %>% 
    mutate(late = arr_delay > 60)

df5 <- late_arr_flights %>%
    select(arr_time, sched_arr_time, arr_delay, carrier, origin, dest, late) %>%
    filter(late == TRUE) %>%
    count(carrier, dest) %>%
    rename(arr_delay_counts = n) %>%
    arrange(desc(arr_delay_counts)) %>%
    left_join(airlines %>%
        rename(carrier_name = name), by = "carrier") %>%
    left_join(airports %>%
        select(faa, name) %>%
        rename(dest_airport_name = name), by = c("dest" = "faa")) %>%
    select(carrier_name, dest_airport_name, arr_delay_counts)

df5 %>% head()
cat("\n")
glue::glue("The airline which had the most of arrival delay flights is {df5[1, 1]} with {df5[1, 3]} flights delayed")

,carrier_name,dest_airport_name,arr_delay_counts
,<chr>,<chr>,<int>
1,Delta Air Lines Inc.,Hartsfield Jackson Atlanta Intl,723
2,JetBlue Airways,Fort Lauderdale Hollywood Intl,646
3,JetBlue Airways,Orlando Intl,633
4,United Air Lines Inc.,Chicago Ohare Intl,597
5,United Air Lines Inc.,San Francisco Intl,492
6,ExpressJet Airlines Inc.,Washington Dulles Intl,468


The airline which had the most of arrival delay flights is Delta Air Lines Inc. with 723 flights delayed

# PosgreSQL Database Connection

In [ ]:
install.packages("RPostgreSQL")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(DBI)
library(RPostgreSQL)
library(dplyr)

In [ ]:
con <- dbConnect(
  PostgreSQL(),
  host = "manny.db.elephantsql.com",
  dbname = "qcjrwjra",
  port = 5432,
  user = "qcjrwjra",
  password = "SfohFEvRbPlaXs9fOqQo71PR0rJOiT-0"
)

con

<PostgreSQLConnection>

In [ ]:
dbWriteTable(con, "top_airline_q3", df2, row.names=FALSE)
dbWriteTable(con, "top_plane_maker", df3, row.names=FALSE)
dbWriteTable(con, "popular_route", df4, row.names=FALSE)
dbWriteTable(con, "top_arr_delay", df5, row.names=FALSE)

In [ ]:
dbListTables(con)

[1] "students"        "top_airline_q3"  "top_plane_maker" "popular_route"  
[5] "top_arr_delay"

In [ ]:
select Name, ROW_NUMBER() OVER (ORDER BY Name DESC) AS ID,
Age, Gender

In [ ]:
glue::glue("Airlines operated in Quarter 3 of 2013 (July - September, 2013)")
dbGetQuery(con, "SELECT * FROM top_airline_q3
                 LIMIT 5")
cat("\n")

glue::glue("The most frequently used manufacturer airplane in 2013, describing the aircraft manufacturer and model")
dbGetQuery(con, "SELECT * FROM top_plane_maker
                 LIMIT 5")
cat("\n")

glue::glue("Ranking of demand route with name of origin and destination airport")                 
rank <- dbGetQuery(con, "SELECT ROW_NUMBER() OVER () AS rank, *
                 FROM popular_route
                 LIMIT 10")
cat("\n")

glue::glue("Airlines and destinations with later than 60-minute arrival")
dbGetQuery(con, "SELECT * FROM top_arr_delay
                 LIMIT 5")

Airlines operated in Quarter 3 of 2013 (July - September, 2013)

,carrier,carrier_name,operate_counts
,<chr>,<chr>,<int>
1,UA,United Air Lines Inc.,14692
2,B6,JetBlue Airways,14113
3,EV,ExpressJet Airlines Inc.,13188
4,DL,Delta Air Lines Inc.,12356
5,AA,American Airlines Inc.,8156


The most frequently used manufacturer airplane in 2013, describing the aircraft manufacturer and model

,model,manufacturer,operated_count
,<chr>,<chr>,<int>
1,A320-232,AIRBUS,30972
2,EMB-145LR,EMBRAER,26475
3,ERJ 190-100 IGW,EMBRAER,23481
4,A320-232,AIRBUS INDUSTRIE,14465
5,737-824,BOEING,13773


Ranking of demand route with name of origin and destination airport

Airlines and destinations with later than 60-minute arrival

,carrier_name,dest_airport_name,arr_delay_counts
,<chr>,<chr>,<int>
1,Delta Air Lines Inc.,Hartsfield Jackson Atlanta Intl,723
2,JetBlue Airways,Fort Lauderdale Hollywood Intl,646
3,JetBlue Airways,Orlando Intl,633
4,United Air Lines Inc.,Chicago Ohare Intl,597
5,United Air Lines Inc.,San Francisco Intl,492


In [ ]:
#CLOSE connection
dbDisconnect(con)

[1] TRUE